<a href="https://colab.research.google.com/github/fernandojunior/deteccao-fraude/blob/master/src/Fraud_Detection_Xente.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Spark load lib

In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('xente').getOrCreate()

In [3]:
!pip install -r requirements.txt
spark

# Load Training Data

In [0]:
import pandas as pd
training_data = pd.read_csv("https://drive.google.com/uc?export=download&id=1NrtVkKv8n_g27w5elq9HWZA1i8aFBW0G")
training_data.head()
df = spark.createDataFrame(training_data)

## Dicionário dos Dados

*   **TransactionId:** Unique transaction identifier on platform.
*   **BatchId:** Unique number assigned to a batch of transactions for processing.
*   **AccountId:** Unique number identifying the customer on platform.
*   **SubscriptionId:** Unique number identifying the customer subscription.
*   **CustomerId:** Unique identifier attached to Account.
*   **CurrencyCode:** Country currency.
*   **CountryCode:** Numerical geographical code of country.
*   **ProviderId:** Source provider of Item bought.
*   **ProductId:** Item name being bought.
*   **ProductCategory:** ProductIds are organized into these broader product categories.
*   **ChannelId:** Identifies if customer used web,Android, IOS, pay later or checkout.
*   **Amount:** Value of the transaction. Positive for debits from customer account and negative for credit into customer account.
*   **Value:** Absolute value of the amount.
*   **TransactionStartTime:** Transaction start time.
*   **PricingStrategy:** Category of Xente's pricing structure for merchants.
*   **FraudResult:** Fraud status of transaction: 1) Yes; or 0) No.

In [6]:
df.printSchema()

root
 |-- TransactionId: string (nullable = true)
 |-- BatchId: string (nullable = true)
 |-- AccountId: string (nullable = true)
 |-- SubscriptionId: string (nullable = true)
 |-- CustomerId: string (nullable = true)
 |-- CurrencyCode: string (nullable = true)
 |-- CountryCode: long (nullable = true)
 |-- ProviderId: string (nullable = true)
 |-- ProductId: string (nullable = true)
 |-- ProductCategory: string (nullable = true)
 |-- ChannelId: string (nullable = true)
 |-- Amount: double (nullable = true)
 |-- Value: long (nullable = true)
 |-- TransactionStartTime: string (nullable = true)
 |-- PricingStrategy: long (nullable = true)
 |-- FraudResult: long (nullable = true)



## Análise dos Dados Faltantes

In [7]:
print(df.count())
df.na.drop(how='any').count()

95662


95662

**Não** há dados faltantes no dataset de **treinamento** : https://zindi.africa/competitions/xente-fraud-detection-challenge/data


### Há linhas duplicadas?



In [8]:
df = df.distinct()
print(df.count())

95662


Como pode ser observado, o número de linhas são iguais, portanto, todas as linhas são distintas.

### Tratamento dos Dados

A oluna "Amount" representa o valor envolvido na operação, entretanto, se esse valor for positivo, isso indica que a transação foi débito, caso contrário, valores negativos representam operação crédito.

Vamos criar uma coluna para indicar se a transação foi crédito (-1.0) ou débito (1.0), e substituir a coluna pelo vetor absoluto de "Amount".


In [40]:
import pyspark.sql.functions as F
df = df.withColumn("Operation", F.when(df.Amount > 0, 1).when(df.Amount < 0, -1).otherwise(0))
df.show()

+--------------------+--------------+--------------+-------------------+---------------+------------+-----------+------------+------------+------------------+-----------+-------+-----+--------------------+---------------+-----------+--------------+---------+
|       TransactionId|       BatchId|     AccountId|     SubscriptionId|     CustomerId|CurrencyCode|CountryCode|  ProviderId|   ProductId|   ProductCategory|  ChannelId| Amount|Value|TransactionStartTime|PricingStrategy|FraudResult|PositiveAmount|Operation|
+--------------------+--------------+--------------+-------------------+---------------+------------+-----------+------------+------------+------------------+-----------+-------+-----+--------------------+---------------+-----------+--------------+---------+
| TransactionId_76871| BatchId_36123|AccountId_3957| SubscriptionId_887|CustomerId_4406|         UGX|        256|ProviderId_6|ProductId_10|           airtime|ChannelId_3| 1000.0| 1000|2018-11-15T02:18:49Z|              2|  

Esses dados apresentam um problema, teoricamente a coluna "Value" representa o valor positivo da coluna "Amount", mas nota-se ligeira diferença em algumas células.



In [43]:
from  pyspark.sql.functions import abs
df = df.withColumn("PositiveAmount", abs(df["Amount"]))
df.drop("Amount")
df.show()

+--------------------+--------------+--------------+-------------------+---------------+------------+-----------+------------+------------+------------------+-----------+-------+-----+--------------------+---------------+-----------+--------------+---------+
|       TransactionId|       BatchId|     AccountId|     SubscriptionId|     CustomerId|CurrencyCode|CountryCode|  ProviderId|   ProductId|   ProductCategory|  ChannelId| Amount|Value|TransactionStartTime|PricingStrategy|FraudResult|PositiveAmount|Operation|
+--------------------+--------------+--------------+-------------------+---------------+------------+-----------+------------+------------+------------------+-----------+-------+-----+--------------------+---------------+-----------+--------------+---------+
| TransactionId_76871| BatchId_36123|AccountId_3957| SubscriptionId_887|CustomerId_4406|         UGX|        256|ProviderId_6|ProductId_10|           airtime|ChannelId_3| 1000.0| 1000|2018-11-15T02:18:49Z|              2|  

## Estatísticas Descritivas 

In [42]:
df.groupBy("FraudResult").count().show()

+-----------+-----+
|FraudResult|count|
+-----------+-----+
|          0|95469|
|          1|  193|
+-----------+-----+



Nota-se um grande desbalanceamento dos dados.

In [46]:
feature_cols = ["CountryCode", "PositiveAmount", "Operation", "Value", "PricingStrategy"]

df.describe(feature_cols).show()
print('Descrição dos dados genuínos:')
df.select(feature_cols).filter(df['FraudResult'] == 0).toPandas().describe()

+-------+-----------+-----------------+------------------+------------------+------------------+
|summary|CountryCode|   PositiveAmount|         Operation|             Value|   PricingStrategy|
+-------+-----------+-----------------+------------------+------------------+------------------+
|  count|      95662|            95662|             95662|             95662|             95662|
|   mean|      256.0|9857.109892120172|0.2015847462942443| 9900.583941376932|  2.25597415901821|
| stddev|        0.0|123095.6235138779|0.9794761961611711|123122.08777619716|0.7329241094855711|
|    min|        256|              1.2|                -1|                 2|                 0|
|    max|        256|        9880000.0|                 1|           9880000|                 4|
+-------+-----------+-----------------+------------------+------------------+------------------+

Descrição dos dados genuínos:


,CountryCode,PositiveAmount,Operation,Value,PricingStrategy
count,95469.0,9.546900e+04,95469.000000,9.546900e+04,95469.000000
mean,256.0,6.723033e+03,0.200075,6.763227e+03,2.257089
std,0.0,3.995819e+04,0.979786,3.999484e+04,0.731971
min,256.0,1.200000e+00,-1.000000,2.000000e+00,0.000000
25%,256.0,2.500000e+02,-1.000000,2.500000e+02,2.000000
50%,256.0,1.000000e+03,1.000000,1.000000e+03,2.000000
75%,256.0,5.000000e+03,1.000000,5.000000e+03,2.000000
max,256.0,2.400000e+06,1.000000,2.400000e+06,4.000000


In [49]:
print('Descrição dos dados fraudulentos:')
df.select(feature_cols).filter(df['FraudResult'] == 1).toPandas().describe()

Descrição dos dados fraudulentos:


,CountryCode,PositiveAmount,Operation,Value,PricingStrategy
count,193.0,1.930000e+02,193.000000,1.930000e+02,193.000000
mean,256.0,1.560153e+06,0.948187,1.561820e+06,1.704663
std,0.0,2.082015e+06,0.318540,2.081227e+06,0.963300
min,256.0,5.000000e+02,-1.000000,5.000000e+02,0.000000
25%,256.0,5.000000e+05,1.000000,5.000000e+05,2.000000
50%,256.0,6.500000e+05,1.000000,6.500000e+05,2.000000
75%,256.0,2.000000e+06,1.000000,2.000000e+06,2.000000
max,256.0,9.880000e+06,1.000000,9.880000e+06,4.000000


Algumas coisas para se observar na distribuição dos dados:

*   Ao menos 25% das operações genuínas são de crédito, enquanto nas fraudes passam de 75% de operações de débito.
*   Em relação ao valor da transação, o valor médio de uma transação genuína é de \$ 6763 (mediana de: \$ 1000), enquanto nas transações fraudulentas o valor médio é de \$ 1.561.820 (mediana de \$ 650.000).
*   O tipo de operação também muda bastante, o valor médio é pelo menos 4 vezes maior em transações fraudulentas.

### Associação Fraude-Transação

In [0]:
group_fraud = df.groupBy('FraudResult')

In [0]:
from pyspark.sql.functions import countDistinct
df.select(countDistinct('FraudResult').alias('I/O')).show()

+---+
|I/O|
+---+
|  2|
+---+



In [0]:
print('Quantity of frauds: ' + str(df.filter(df['FraudResult'] == 1).count()) )

example_fraud = df.filter(df['FraudResult'] == 1).collect()
example_fraud[0]

Quantity of frauds: 193


Row(TransactionId='TransactionId_60873', BatchId='BatchId_93685', AccountId='AccountId_3832', SubscriptionId='SubscriptionId_4161', CustomerId='CustomerId_4275', CurrencyCode='UGX', CountryCode=256, ProviderId='ProviderId_3', ProductId='ProductId_15', ProductCategory='financial_services', ChannelId='ChannelId_3', Amount=700000.0, Value=700000, TransactionStartTime='2018-11-15T08:04:51Z', PricingStrategy=0, FraudResult=1)

In [0]:
#TransactionId = TransactionId_76871
#TransactionId = 76871
example_fraud.split(df['TransactionId'], '_')[1]

## Análise de Normalidade

## Análise e Tratamento de Outliers

## Análise de Variância


## Análise de Correlação

## Análise e Tratamento dos dados desbalanceados

# Preparação dos Dados

## Conversão dos dados Categóricos

## Normalização dos Dados

## Extração de características

# Modelagem

# Avaliação


In [0]:
import pandas as pd
test_data = pd.read_csv("https://drive.google.com/uc?export=download&id=16cRQIFW6n2th2YOK7DEsp9dQgihHDuHa")
test_data.head()

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy
0,TransactionId_50600,BatchId_35028,AccountId_2441,SubscriptionId_4426,CustomerId_2857,UGX,256,ProviderId_5,ProductId_3,airtime,ChannelId_3,1000.0,1000,2019-02-13T10:01:40Z,4
1,TransactionId_95109,BatchId_45139,AccountId_3439,SubscriptionId_2643,CustomerId_3874,UGX,256,ProviderId_5,ProductId_15,financial_services,ChannelId_3,2000.0,2000,2019-02-13T10:02:12Z,2
2,TransactionId_47357,BatchId_74887,AccountId_4841,SubscriptionId_3829,CustomerId_2857,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-50.0,50,2019-02-13T10:02:30Z,2
3,TransactionId_28185,BatchId_11025,AccountId_2685,SubscriptionId_4626,CustomerId_3105,UGX,256,ProviderId_5,ProductId_10,airtime,ChannelId_3,3000.0,3000,2019-02-13T10:02:38Z,4
4,TransactionId_22140,BatchId_29804,AccountId_4841,SubscriptionId_3829,CustomerId_3105,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-60.0,60,2019-02-13T10:02:58Z,2


### Análise de dados 

In [0]:
df_test = spark.createDataFrame(test_data)
print(df_test.count())
df_test.na.drop(how='any').count()

45019


45019